In [61]:
import pandas as pd
from datetime import datetime

In [62]:
df = pd.read_csv("incidentes_viales (2).csv", sep = ";",  low_memory = False , encoding = 'utf_8',encoding_errors = 'ignore')


In [63]:
df.head()

,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DIRECCION ENCASILLADA,DISEÑO,EXPEDIENTE,FECHA_ACCIDENTE,FECHA_ACCIDENTES,MES,NRO_RADICADO,NUMCOMUNA,BARRIO,COMUNA,LOCATION,X,Y
0,2014,1105,Otro,CR 66 B CL 34,CR 066 B 034 000 00000,Tramo de via,A000095088,05/08/2014 07:15:00,2014-08-05T12:15:00.000Z,Con heridos,8,1451888,11,Los Conquistadores,Laureles Estadio,"[-75.589313202, 6.2397866157]",832677.92
1,2014,0509,Choque,CR 71 CL 103 D,CR 071 103 D 000 00000,Interseccion,A000095143,05/08/2014 14:30:00,2014-08-05T19:30:00.000Z,Con heridos,8,1451891,05,Girardot,Castilla,"[-75.5708179064, 6.3000411778]",834744.36
2,2014,1421,Choque,CR 49 CL 7,CR 049 007 000 00000,Tramo de via,A000095226,05/08/2014 15:50:00,2014-08-05T20:50:00.000Z,Con heridos,8,1451935,14,Patio Bonito,El Poblado,"[-75.5765743781, 6.20874755798]",834078.39
3,2014,0801,Otro,CR 39 A CL 66 C,CR 039 A 066 C 000 00000,Tramo de via,A000095227,05/08/2014 13:00:00,2014-08-05T18:00:00.000Z,Con heridos,8,1451956,08,Villa Hermosa,Villa Hermosa,"[-75.5512546684, 6.25905761463]",836897.02
4,2014,1017,Choque,CL 58 CR 37,CL 058 037 000 00000,Tramo de via,A000094940,05/08/2014 19:30:00,2014-08-06T00:30:00.000Z,Con heridos,8,1452022,10,Los Ángeles,La Candelaria,"[-75.5561495119, 6.24996631944]",836352.35


In [64]:
df.rename({'MES': 'GRAVEDAD','NRO_RADICADO':'MES','NUMCOMUNA':'NRO_RADICADO','BARRIO':'NUMCOMUNA','COMUNA':'BARRIO','LOCATION':'COMUNA','X': 'LOCATION','Y':'X'}, axis = 1, inplace = True)# Se reorganizan los nombres de la columna pues hay un
# problema en la descarga de los datos.
# OJO SOLO CORRER UNA VEZ

In [65]:
df.isna().sum()

AÑO                          0
CBML                     18156
CLASE_ACCIDENTE              6
DIRECCION                    0
DIRECCION ENCASILLADA      391
DISEÑO                    1148
EXPEDIENTE                 110
FECHA_ACCIDENTE              0
FECHA_ACCIDENTES             0
GRAVEDAD                     0
MES                          0
NRO_RADICADO                 5
NUMCOMUNA                    0
BARRIO                   19006
COMUNA                   12798
LOCATION                     0
X                            0
dtype: int64

## Eliminación de Variables

Se eliminan las siguientes variables. Expediente = Es un registro aleatorio sin análiss, NRo _ Radicao = Numero aleatorio, Direccion Encasillada = Se repite, FECHA_ACCIDENTES = Se repite, COMUNA = se tumba comuna pues tiene mucho nan y existe otra columna con la misma información , CBML = Mucha inconsistencia en caso de requerir el codigo catastral se puede buscar una página externa que reciba el código  catastraly entregue la localización , X = Debido a que existe un problema en la descarga de la base de datos no existe datos de la variable 'y' entonces es necesario eliminar la componente 'x' pues en ausencia de 'y', 'x' pierde sentido.
Se eliminara la dirección pues tampoco hay una nomeclatura clara. Verificar luego.

In [66]:

df.drop(['EXPEDIENTE','DIRECCION','NRO_RADICADO','DIRECCION ENCASILLADA', 'FECHA_ACCIDENTES','COMUNA','CBML','X'],axis = 1 , inplace = True)

In [67]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             19006
LOCATION               0
dtype: int64

Debido a que existe un problema en la descarga de la base de datos no existe datos de la variable 'y' entonces es necesario eliminar la componente 'x' pues en ausencia de 'y', 'x' pierde sentido

In [68]:
df.shape

(270765, 9)

In [69]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             19006
LOCATION               0
dtype: int64

In [70]:
df.columns

Index(['AÑO', 'CLASE_ACCIDENTE', 'DISEÑO', 'FECHA_ACCIDENTE', 'GRAVEDAD',
       'MES', 'NUMCOMUNA', 'BARRIO', 'LOCATION'],
      dtype='object')

In [71]:
df['GRAVEDAD'].unique()

array(['Con heridos', 'Solo daños', 'Con muertos', 'Solo da\\xF1os'],
      dtype=object)

In [72]:
df['AÑO']= df['AÑO'].apply(lambda x : '2019' if x == '2019\\r' else x)
df['DISEÑO'] = df['DISEÑO'].apply(lambda x : 'Pontón' if x == 'Pont\\xF3n' else x)
df['HORA'] = df['FECHA_ACCIDENTE'].apply(lambda x: x.split(' ')[1])
df['FECHA_ACCIDENTE'] = df['FECHA_ACCIDENTE'].apply(lambda x: x.split(' ')[0])
df['GRAVEDAD'] = df['GRAVEDAD'].apply(lambda x: 'Solo daños' if x == 'Solo da\\xF1os' else x)
df['CLASE_ACCIDENTE'] = df['CLASE_ACCIDENTE'].apply(lambda x : 'Caída de Ocupante' if x[0] =='C' else x)

Para el número de comuna hay muchos datos que no son consistentes ('0','Sin Inf','In','SN','AU'). Se plantea la posibilidad de colocar 99 que corresponde a toda la ciudad es decir en cualquier parte o que no hay información. Se espera que con la localización se puede precisar

In [73]:
len(df[df['NUMCOMUNA'].isin(['0','Sin Inf','In','SN','AU'])])

20127

In [74]:
df['NUMCOMUNA'] = df['NUMCOMUNA'].apply(lambda x: '99' if x in(['0','Sin Inf','In','SN','AU']) else x)

In [75]:
def encondebarrio(x):
    
    z = {'Bel\\xE9n' : 'Belén', 'Belalc\\xE1zar' : 'Belalcázar','H\\xE9ctor Abad G\\xF3mez': 'Héctor Abad Gómez',
        'San Joaqu\\xEDn' : 'San Joaquín', 'Santa F\\xE9' : 'Santa Fé', 'Coraz\\xF3n de Jes\\xFAs' : 'Corazón de Jesús',
        'Alfonso L\\xF3pez': 'Alfonso López', 'Santa Mar\\xEDa de Los \\xC1ngeles' : 'Santa María de Los Ángeles',
        'El Rinc\\xF3n' : 'El Rincón', 'Alejandr\\xEDa': 'Alejandría', 'Campo Vald\\xE9s No. 1' : 'Campo Valdés No. 1',
        'Barrio Col\\xF3n' : 'Barrio Colón','Jard\\xEDn Bot\\xE1nico' : 'Jardín Botánico', 'Estaci\\xF3n Villa' : 'Estación Villa',
       'Barrio de Jes\\xFAs' : 'Barrio de Jesús', 'Barrio Crist\\xF3bal': 'Barrio Cristóbal', 'Boyac\\xE1' : 'Boyacá',
       'Mosc\\xFA No. 1':'Moscú No. 1', 'Sim\\xF3n Bol\\xEDvar' : 'Simón Bolívar', 'Nueva Villa de la Iguan\\xE1' : 'Nueva Villa de la Iguaná',
        'Bombon\\xE1 No. 2': 'Bomboná No. 2', 'La Pi\\xF1uela' : 'La Piñuela' , 'Play\\xF3n de Los Comuneros': 'Playón de Los Comuneros',
       'Jes\\xFAs Nazareno' : 'Jesús Nazareno', '\\xC1rea de Expansi\\xF3n Pajarito': 'Área de Expansión Pajarito', 'F\\xE1tima':'Fátima',
        'San Germ\\xE1n' : 'San Germán', 'Diego Echavarr\\xEDa':'Diego Echavarría', 'Mosc\\xFA No. 2':'Moscú No. 2', 'La Am\\xE9rica': 'La América',
        'Alejandro Echavarr\\xEDa':'Alejandro Echavarría','L\\xF3pez de Mesa':'López de Mesa', 'Los \\xC1ngeles' : 'Los Ángeles',
       '\\xC1rea de Expansi\\xF3n Altos de Calasanz':'Área de Expansión Altos de Calasanz' ,'Santa Mar\\xEDa de los \\xC1ngeles' : 'Santa María de los Ángeles', 
        'Santa M\\xF3nica': 'Santa Mónica', 'La Mansi\\xF3n': 'La Mansión', 'San Mart\\xEDn de Porres':'San Martín de Porres', 'C\\xF3rdoba': 'Córdoba',
       'Santa Luc\\xEDa': 'Santa Lucía', 'Batall\\xF3n Girardot': 'Batallón Girardot', 'El Vel\\xF3dromo':'El Velódromo', 'Andaluc\\xEDa': 'Andalucía', 
        'Bombon\\xE1 No.1' :'Bomboná No.1' , 'Mar\\xEDa Cano Carambolas': 'María Cano Carambolas', 'Nueva Villa de Aburr\\xE1' : 'Nueva Villa de Aburrá',
       '\\xC1rea de Expansi\\xF3n San Antonio de Prado': 'Área de Expansión San Antonio de Prado','Los Alc\\xE1zares' : 'Los Alcázares', 'El Coraz\\xF3n': 'El Corazón',
        'Catalu\\xF1a' : 'Cataluña', 'Berl\\xEDn': 'Berlín', 'Antonio Nari\\xF1o' : 'Antonio Nariño', 'Hospital San Vicente de Pa\\xFAl': 'Hospital San Vicente de Paúl',
       'San Jos\\xE9 de la Monta\\xF1a' : 'San José de la Montaña', 'Campo Vald\\xE9s No.2' : 'Campo Valdés No.2', 'San Jos\\xE9 la Cima No.2': 'San José la Cima No.2',
        'Mosc\\xFA No.2' : 'Moscú No. 2', 'San Jos\\xE9 la Cima No. 1': 'San José la Cima No. 1', 'Bombon\\xE1 No. 1' : 'Bomboná No. 1', 'Santa In\\xE9s': 'Santa Inés',
        'Campo Vald\\xE9s No. 2' : 'Campo Valdés No. 2'}
    
    
    if x in z:
        x = z[x]
    elif type(x) == float :
        x = None
    elif x[0] in ['0','1','2','3','4','5','6','7','8','9']:
        x = None
    else:
        x = x
    return x

In [76]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             19006
LOCATION               0
HORA                   0
dtype: int64

In [77]:
df['BARRIO'] = df['BARRIO'].apply(encondebarrio)

In [78]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             20613
LOCATION               0
HORA                   0
dtype: int64

Con las siguientes agrupaciones se puede inferir si es que tanta información se pierde. Podemos hacer como una distribución de los nan y los que no son nan

In [79]:
df_datos = df.groupby(['AÑO','MES'])[['BARRIO']].agg([ lambda x : x.isnull().sum(), lambda x : x.notnull().sum()])
df_datos.rename({df_datos.columns[0][1]:'Data perdida',df_datos.columns[1][1] : 'Datos completos'}, axis = 1, inplace = True)
df_datos.head(20)

BARRIO                
         Data perdida Datos completos
AÑO  MES                             
2014 7            231            3189
     8            307            3633
     9            290            3587
     10           266            3470
     11           275            3337
     12           292            3538
2015 1            283            2980
     2            226            3355
     3            329            3667
     4            274            3472
     5            348            3706
     6            287            3312
     7            290            3771
     8            329            3775
     9            294            3841
     10           268            3708
     11           290            3434
     12           279            3654
2016 1            241            3081
     2            289            3569

In [80]:
df_Tipochoque = df.groupby(['AÑO','CLASE_ACCIDENTE'])[['CLASE_ACCIDENTE']].count()
df_Tipochoque.head(20)

CLASE_ACCIDENTE
AÑO  CLASE_ACCIDENTE                   
2014 Atropello                     2528
     Caída de Ocupante             2299
     Choque                       14440
     Incendio                         4
     Otro                          2588
     Volcamiento                    556
2015 Atropello                     4851
     Caida Ocupante                4239
     Caida de Ocupante                1
     Caída de Ocupante               21
     Choque                       30500
     Incendio                         1
     Otro                          4876
     Volcamiento                   1682
2016 Atropello                     4464
     Caída de Ocupante             4189
     Choque                       31128
     Incendio                         4
     Otro                          5566
     Volcamiento                   1747

In [81]:
df_Gravedad = df.groupby(['AÑO','GRAVEDAD'])[['GRAVEDAD']].count()
df_Gravedad

GRAVEDAD
AÑO  GRAVEDAD             
2014 Con heridos     12584
     Con muertos       140
     Solo daños       9691
2015 Con heridos     25911
     Con muertos       266
     Solo daños      19995
2016 Con heridos     26597
     Con muertos       260
     Solo daños      20246
2017 Con heridos     25184
     Con muertos       249
     Solo daños      20772
2018 Con heridos     22835
     Con muertos       233
     Solo daños      20290
2019 Con heridos     25001
     Con muertos       234
     Solo daños      20560
2020 Con heridos     11959
     Con muertos       107
     Solo daños       7651

In [82]:
df_ByMES = df.groupby(['AÑO','MES'])[['MES']].count()
df_ByMES.rename({df_ByMES.columns[0]:'Total'}, axis = 1, inplace = True)
df_ByMES

Total
AÑO  MES       
2014 7     3420
     8     3940
     9     3877
     10    3736
     11    3612
...         ...
2020 4      898
     5     1741
     6     2191
     7     2152
     8     2270

[74 rows x 1 columns]

In [83]:
df_Bycomuna = df.groupby(['AÑO','NUMCOMUNA'])[['NUMCOMUNA']].count()
df_Bycomuna.rename({df_Bycomuna.columns[0]:'Total'}, axis = 1, inplace = True)
df_Bycomuna

Total
AÑO  NUMCOMUNA       
2014 01           315
     02           328
     03           676
     04          1139
     05          2064
...               ...
2020 8            675
     80           203
     9            863
     90            89
     99          1258

[153 rows x 1 columns]

In [84]:
df_BymesCOM = df.groupby(['AÑO','MES','NUMCOMUNA'])[['NUMCOMUNA']].count()
df_BymesCOM.rename({df_BymesCOM.columns[0]:'Total'}, axis = 1, inplace = True)
df_BymesCOM

Total
AÑO  MES NUMCOMUNA       
2014 7   01            44
         02            47
         03            92
         04           181
         05           280
...                   ...
2020 8   8             85
         80            35
         9             89
         90            11
         99           139

[1570 rows x 1 columns]

In [85]:
df_ByDis = df.groupby(['AÑO','DISEÑO'])[['DISEÑO']].count()
df_ByDis.rename({df_ByDis.columns[0]:'Total'}, axis = 1, inplace = True)
df_ByDis

Total
AÑO  DISEÑO              
2014 Ciclo Ruta       189
     Glorieta         376
     Interseccion    2965
     Lote o Predio    723
     Paso Elevado      61
...                   ...
2020 Pontón             2
     Puente            73
     Tramo de via   11835
     Tunel             18
     Via peatonal       3

[84 rows x 1 columns]

In [86]:
df['HORA'] = df['HORA'].apply(lambda x: None if x == 'Sin' or x == '' else x)

In [87]:
df['HORA'] = pd.to_datetime(df['HORA'],format= '%H:%M:%S' ).dt.time
df['FECHA_ACCIDENTE'] = pd.to_datetime(df['FECHA_ACCIDENTE'], format="%d/%m/%Y")

In [88]:
def momentodia(x):
    
    H1 = datetime.strptime('06:00:00', "%X").time()
    H2 = datetime.strptime('12:00:00', "%X").time()
    H3 = datetime.strptime('18:00:00', "%X").time()
    H4 = datetime.strptime('23:59:59', "%X").time()
    
    if type(x) != type(H1):
        return None
    elif x >= H1 and x < H2:
        return 'Mañana'
    elif x >= H2 and x < H3:
        return 'Tarde'
    elif x >= H3 and x < H4:
        return 'Noche'
    else:
        return 'Madrugada'
    

In [89]:
df['MOMENTO'] = df['HORA'].apply(momentodia)

In [91]:
df_Momentodia = df.groupby(['AÑO','MOMENTO'])[['MOMENTO']].count()
df_Momentodia.rename({df_Momentodia.columns[0]:'Total'}, axis = 1, inplace = True)
df_Momentodia

Total
AÑO  MOMENTO         
2014 Madrugada   1514
     Mañana      6794
     Noche       5609
     Tarde       8498
2015 Madrugada  11614
     Mañana     20811
     Noche       4261
     Tarde       9486
2016 Madrugada   2983
     Mañana     14551
     Noche      11480
     Tarde      18087
2017 Madrugada   2908
     Mañana     14097
     Noche      11346
     Tarde      17850
2018 Madrugada   2740
     Mañana     13135
     Noche      10613
     Tarde      16870
2019 Madrugada   3076
     Mañana     14164
     Noche      10741
     Tarde      17814
2020 Madrugada   1430
     Mañana      5924
     Noche       4427
     Tarde       7936

Luego de haber justificado la eliminación los datos faltantes en el barrio con el proceso descriptivo. Tumbamos los datos.

In [95]:
df.shape

(270765, 11)

In [96]:
df.dropna(inplace = True)

In [97]:
df.shape

(249083, 11)

In [105]:
df['CLASE_ACCIDENTE'].unique()

array(['Otro', 'Caída de Ocupante', 'Atropello', 'Volcamiento',
       'Incendio'], dtype=object)

In [101]:
df.head()

,AÑO,CLASE_ACCIDENTE,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD,MES,NUMCOMUNA,BARRIO,LOCATION,HORA,MOMENTO
0,2014,Otro,Tramo de via,2014-08-05,Con heridos,8,11,Los Conquistadores,"[-75.589313202, 6.2397866157]",07:15:00,Mañana
1,2014,Choque,Interseccion,2014-08-05,Con heridos,8,05,Girardot,"[-75.5708179064, 6.3000411778]",14:30:00,Tarde
2,2014,Choque,Tramo de via,2014-08-05,Con heridos,8,14,Patio Bonito,"[-75.5765743781, 6.20874755798]",15:50:00,Tarde
3,2014,Otro,Tramo de via,2014-08-05,Con heridos,8,08,Villa Hermosa,"[-75.5512546684, 6.25905761463]",13:00:00,Tarde
4,2014,Choque,Tramo de via,2014-08-05,Con heridos,8,10,Los Ángeles,"[-75.5561495119, 6.24996631944]",19:30:00,Noche
